In [42]:
import glob
import math
import matplotlib.pyplot as plt
import multiprocessing
import numpy as np
import openslide
from openslide import OpenSlideError
import os
import PIL
from PIL import Image, ImageDraw, ImageFont
import re
import sys
import numpy as np
import scipy.ndimage.morphology as sc_morph
import skimage.color as sk_color
import skimage.exposure as sk_exposure
import skimage.feature as sk_feature
import skimage.filters as sk_filters
import skimage.future as sk_future
import skimage.morphology as sk_morphology
import skimage.segmentation as sk_segmentation
from enum import Enum
import colorsys

In [93]:
np.random.seed(0)
BASE_DIR = os.path.join(".","tcga_images")
SLIDE_IMG= os.path.join(".","tcga_images","TCGA-3P-A9WA-01A-01-TS1.EC84245E-56A9-46C1-9496-CCF2587D318E.svs")
SRC_TRAIN_DIR = os.path.join(".","training_png")
SCALE_FACTOR = 32
DEST_TRAIN_EXT = "png"
SRC_TRAIN_EXT = "svs"
DEST_TRAIN_DIR = os.path.join("training_" + DEST_TRAIN_EXT)

FILTER_RESULT_TEXT = "filtered"
FILTER_DIR = os.path.join("filter_" + DEST_TRAIN_EXT)
FILTER_PAGINATION_SIZE = 50
FILTER_PAGINATE = True

TILE_SUMMARY_DIR = os.path.join(".","tile_summary_" + DEST_TRAIN_EXT)
TILE_SUMMARY_PAGINATION_SIZE = 50
TILE_SUMMARY_PAGINATE = True
TILE_SUMMARY_HTML_DIR = BASE_DIR

TILE_DATA_DIR = os.path.join(".", "tile_data")
TILE_DATA_SUFFIX = "tile_data"
TILE_DIR = os.path.join(".", "tile_images")
TILE_DIR

'./tile_images'

In [32]:
TISSUE_HIGH_THRESH = 80
TISSUE_LOW_THRESH = 10

ROW_TILE_SIZE = 1024
COL_TILE_SIZE = 1024
NUM_TOP_TILES = 50

DISPLAY_TILE_SUMMARY_LABELS = False
TILE_LABEL_TEXT_SIZE = 10
LABEL_ALL_TILES_IN_TOP_TILE_SUMMARY = False
BORDER_ALL_TILES_IN_TOP_TILE_SUMMARY = False

TILE_BORDER_SIZE = 2  

HIGH_COLOR = (0, 255, 0)
MEDIUM_COLOR = (255, 255, 0)
LOW_COLOR = (255, 165, 0)
NONE_COLOR = (255, 0, 0)

FADED_THRESH_COLOR = (128, 255, 128)
FADED_MEDIUM_COLOR = (255, 255, 128)
FADED_LOW_COLOR = (255, 210, 128)
FADED_NONE_COLOR = (255, 128, 128)

FONT_PATH = "/Library/Fonts/Arial Bold.ttf"
SUMMARY_TITLE_FONT_PATH = "/Library/Fonts/Courier New Bold.ttf"
SUMMARY_TITLE_TEXT_COLOR = (0, 0, 0)
SUMMARY_TITLE_TEXT_SIZE = 24
SUMMARY_TILE_TEXT_COLOR = (255, 255, 255)
TILE_TEXT_COLOR = (0, 0, 0)
TILE_TEXT_SIZE = 36
TILE_TEXT_BACKGROUND_COLOR = (255, 255, 255)
TILE_TEXT_W_BORDER = 5
TILE_TEXT_H_BORDER = 4

HSV_PURPLE = 270
HSV_PINK = 330

In [4]:
files = os.listdir(BASE_DIR)
files[5]

'TCGA-13-2071-01A-02-BS2.f6520f68-3a61-4799-8ab1-6998727698e2.svs'

In [5]:
def get_train_prefix(slide_num):
    strObj = files[slide_num]
    start = 23
    stop = len(files[slide_num])
    strObj = strObj[0: start:] + strObj[stop + 1::]
    return strObj

In [6]:
def open_slide(slide_num):
    try:
        slide = openslide.open_slide(os.path.join(BASE_DIR,files[slide_num]))
    except OpenSlideError:
        slide = print("Cannot open slide")
    except FileNotFoundError:
        slide = print("File not found")
    return slide

In [7]:
def slide_to_scaled_pil_image(slide_num):
        slide = open_slide(slide_num)
        large_w, large_h = slide.dimensions
        new_w = math.floor(large_w / SCALE_FACTOR)
        new_h = math.floor(large_h / SCALE_FACTOR)
        level = slide.get_best_level_for_downsample(SCALE_FACTOR)
        whole_slide_image = slide.read_region((0, 0), level, slide.level_dimensions[level])
        whole_slide_image = whole_slide_image.convert("RGB")
        img = whole_slide_image.resize((new_w, new_h), PIL.Image.BILINEAR)
        return img, large_w, large_h, new_w, new_h

In [8]:
def training_slide_to_image(slide_num):
    img, large_w, large_h, new_w, new_h = slide_to_scaled_pil_image(slide_num)
    img_path = os.path.join(DEST_TRAIN_DIR, get_train_prefix(slide_num) + "." + DEST_TRAIN_EXT)
    if not os.path.exists(DEST_TRAIN_DIR):
        os.makedirs(DEST_TRAIN_DIR)
    img.save(img_path)
    print("Saving image to: " + img_path)

In [9]:
def training_slide_range_to_images(start_ind, end_ind):
    for slide_num in range(start_ind, end_ind):
        training_slide_to_image(slide_num)
    return (start_ind, end_ind)

In [10]:
def multiprocess_training_slides_to_images():
    num_processes = multiprocessing.cpu_count()
    pool = multiprocessing.Pool(num_processes)

    num_train_images = len(files)
    if num_processes > num_train_images:
        num_processes = num_train_images
    images_per_process = num_train_images / num_processes

    print("Number of processes: " + str(num_processes))
    print("Number of training images: " + str(num_train_images))

    tasks = []
    for num_process in range(1, num_processes + 1):
        start_index = (num_process - 1) * images_per_process 
        end_index = num_process * images_per_process 
        start_index = int(start_index)
        end_index = int(end_index)
        tasks.append((start_index, end_index))
        if start_index == end_index:
            print("Task #" + str(num_process) + ": Process slide " + str(start_index))
        else:
            print("Task #" + str(num_process) + ": Process slides " + str(start_index) + " to " + str(end_index))

    results = []
    for t in tasks:
        results.append(pool.apply_async(training_slide_range_to_images, t))
    
    for result in results:
        (start_ind, end_ind) = result.get()
        if start_ind == end_ind:
            print("Done converting slide %d" % start_ind)
        else:
            print("Done converting slides %d through %d" % (start_ind, end_ind))

In [11]:
def get_training_image_path(slide_num):
    training_pngs = os.path.join(SRC_TRAIN_DIR, get_train_prefix(slide_num) + "." + DEST_TRAIN_EXT)
    return training_pngs

In [12]:
def open_image(filename):
    image = Image.open(filename)
    return image

In [13]:
def pil_to_np_rgb(pil_img):
    rgb = np.asarray(pil_img)
    return rgb

In [14]:
def open_image_np(filename):
    pil_img = open_image(filename)
    np_img = pil_to_np_rgb(pil_img)
    return np_img

In [15]:
def save_filtered_image(np_img, slide_num):
    filepath = get_filter_image_path(slide_num)
    pil_img = np_to_pil(np_img)
    pil_img.save(filepath)
  

In [16]:
def mask_percentage_text(mask_percentage):
    return "%3.2f%%" % mask_percentage

In [17]:
def save_display(save, display, info, np_img, slide_num, filter_num, display_text, file_text,display_mask_percentage=True):
    mask_percentage = None
    if display_mask_percentage:
        mask_percentage = mask_percent(np_img)
        display_text = display_text + "\n(" + mask_percentage_text(mask_percentage) + " masked)"
    if slide_num is None and filter_num is None:
        pass
    elif filter_num is None:
        display_text = "S%03d " % slide_num + display_text
    elif slide_num is None:
        display_text = "F%03d " % filter_num + display_text
    else:
        display_text = "S%03d-F%03d " % (slide_num, filter_num) + display_text
    if display:
        display_img(np_img, display_text)
    if save:
        save_filtered_image(np_img, slide_num)
    if info is not None:
        info[slide_num * 1000 + filter_num] = (slide_num, filter_num, display_text, file_text, mask_percentage)

In [18]:
def np_to_pil(np_img):
    if np_img.dtype == "bool":
        np_img = np_img.astype("uint8") * 255
    elif np_img.dtype == "float64":
        np_img = (np_img * 255).astype("uint8")
    return Image.fromarray(np_img)

In [19]:
def display_img(np_img, text=None, font_path="/Library/Fonts/Arial Bold.ttf", size=48, color=(255, 0, 0),background=(255, 255, 255), border=(0, 0, 0), bg=False):
    result = np_to_pil(np_img)
    if result.mode == 'L':
        result = result.convert('RGB')
        draw = ImageDraw.Draw(result)
    if text is not None:
        font = ImageFont.truetype(font_path, size)
    if bg:
        (x, y) = draw.textsize(text, font)
        draw.rectangle([(0, 0), (x + 5, y + 4)], fill=background, outline=border)
        draw.text((2, 0), text, color, font=font)
    result.show()

In [20]:
def mask_percent(np_img):
    if (len(np_img.shape) == 3) and (np_img.shape[2] == 3):
        np_sum = np_img[:, :, 0] + np_img[:, :, 1] + np_img[:, :, 2]
        mask_percentage = 100 - np.count_nonzero(np_sum) / np_sum.size * 100
    else:
        mask_percentage = 100 - np.count_nonzero(np_img) / np_img.size * 100
    return mask_percentage

In [21]:
def filter_grays(rgb, tolerance=15, output_type="bool"):
    (h, w, c) = rgb.shape
    rgb = rgb.astype(np.int)
    rg_diff = abs(rgb[:, :, 0] - rgb[:, :, 1]) <= tolerance
    rb_diff = abs(rgb[:, :, 0] - rgb[:, :, 2]) <= tolerance
    gb_diff = abs(rgb[:, :, 1] - rgb[:, :, 2]) <= tolerance
    result = ~(rg_diff & rb_diff & gb_diff)

    if output_type == "bool":
        pass
    elif output_type == "float":
        result = result.astype(float)
    else:
        result = result.astype("uint8") * 255
    return result

In [22]:
def filter_red(rgb, red_lower_thresh, green_upper_thresh, blue_upper_thresh, output_type="bool",display_np_info=False):
    if display_np_info:
        t = Time()
    r = rgb[:, :, 0] > red_lower_thresh
    g = rgb[:, :, 1] < green_upper_thresh
    b = rgb[:, :, 2] < blue_upper_thresh
    result = ~(r & g & b)
    if output_type == "bool":
        pass
    elif output_type == "float":
        result = result.astype(float)
    else:
        result = result.astype("uint8") * 255
    if display_np_info:
        np_info(result, "Filter Red", t.elapsed())
    return result

In [82]:
def filter_red_pen(rgb, output_type="bool"):
    result = filter_red(rgb, red_lower_thresh=150, green_upper_thresh=80, blue_upper_thresh=90) & \
               filter_red(rgb, red_lower_thresh=110, green_upper_thresh=20, blue_upper_thresh=30) & \
               filter_red(rgb, red_lower_thresh=185, green_upper_thresh=65, blue_upper_thresh=105) & \
               filter_red(rgb, red_lower_thresh=195, green_upper_thresh=85, blue_upper_thresh=125) & \
               filter_red(rgb, red_lower_thresh=220, green_upper_thresh=115, blue_upper_thresh=145) & \
               filter_red(rgb, red_lower_thresh=125, green_upper_thresh=40, blue_upper_thresh=70) & \
               filter_red(rgb, red_lower_thresh=200, green_upper_thresh=120, blue_upper_thresh=150) & \
               filter_red(rgb, red_lower_thresh=100, green_upper_thresh=50, blue_upper_thresh=65) & \
               filter_red(rgb, red_lower_thresh=85, green_upper_thresh=25, blue_upper_thresh=45)
    if output_type == "bool":
        pass
    elif output_type == "float":
        result = result.astype(float)
    else:
        result = result.astype("uint8") * 255
    return result

In [91]:

def filter_green(rgb, red_upper_thresh, green_lower_thresh, blue_lower_thresh, output_type="bool",display_np_info=False):
    if display_np_info:
        t = Time()
    r = rgb[:, :, 0] < red_upper_thresh
    g = rgb[:, :, 1] > green_lower_thresh
    b = rgb[:, :, 2] > blue_lower_thresh
    result = ~(r & g & b)
    if output_type == "bool":
        pass
    elif output_type == "float":
        result = result.astype(float)
    else:
        result = result.astype("uint8") * 255
    if display_np_info:
        np_info(result, "Filter Green", t.elapsed())
    return result

In [83]:
def filter_green_pen(rgb, output_type="bool"):
    result = filter_green(rgb, red_upper_thresh=150, green_lower_thresh=160, blue_lower_thresh=140) & \
               filter_green(rgb, red_upper_thresh=70, green_lower_thresh=110, blue_lower_thresh=110) & \
               filter_green(rgb, red_upper_thresh=45, green_lower_thresh=115, blue_lower_thresh=100) & \
               filter_green(rgb, red_upper_thresh=30, green_lower_thresh=75, blue_lower_thresh=60) & \
               filter_green(rgb, red_upper_thresh=195, green_lower_thresh=220, blue_lower_thresh=210) & \
               filter_green(rgb, red_upper_thresh=225, green_lower_thresh=230, blue_lower_thresh=225) & \
               filter_green(rgb, red_upper_thresh=170, green_lower_thresh=210, blue_lower_thresh=200) & \
               filter_green(rgb, red_upper_thresh=20, green_lower_thresh=30, blue_lower_thresh=20) & \
               filter_green(rgb, red_upper_thresh=50, green_lower_thresh=60, blue_lower_thresh=40) & \
               filter_green(rgb, red_upper_thresh=30, green_lower_thresh=50, blue_lower_thresh=35) & \
               filter_green(rgb, red_upper_thresh=65, green_lower_thresh=70, blue_lower_thresh=60) & \
               filter_green(rgb, red_upper_thresh=100, green_lower_thresh=110, blue_lower_thresh=105) & \
               filter_green(rgb, red_upper_thresh=165, green_lower_thresh=180, blue_lower_thresh=180) & \
               filter_green(rgb, red_upper_thresh=140, green_lower_thresh=140, blue_lower_thresh=150) & \
               filter_green(rgb, red_upper_thresh=185, green_lower_thresh=195, blue_lower_thresh=195)
    if output_type == "bool":
        pass
    elif output_type == "float":
        result = result.astype(float)
    else:
        result = result.astype("uint8") * 255
    return result

In [89]:
def filter_blue(rgb, red_upper_thresh, green_upper_thresh, blue_lower_thresh, output_type="bool",display_np_info=False):
    if display_np_info:
        t = Time()
    r = rgb[:, :, 0] < red_upper_thresh
    g = rgb[:, :, 1] < green_upper_thresh
    b = rgb[:, :, 2] > blue_lower_thresh
    result = ~(r & g & b)
    if output_type == "bool":
        pass
    elif output_type == "float":
        result = result.astype(float)
    else:
        result = result.astype("uint8") * 255
    if display_np_info:
        np_info(result, "Filter Blue", t.elapsed())
    return result

In [84]:
def filter_blue_pen(rgb, output_type="bool"):
    result = filter_blue(rgb, red_upper_thresh=60, green_upper_thresh=120, blue_lower_thresh=190) & \
               filter_blue(rgb, red_upper_thresh=120, green_upper_thresh=170, blue_lower_thresh=200) & \
               filter_blue(rgb, red_upper_thresh=175, green_upper_thresh=210, blue_lower_thresh=230) & \
               filter_blue(rgb, red_upper_thresh=145, green_upper_thresh=180, blue_lower_thresh=210) & \
               filter_blue(rgb, red_upper_thresh=37, green_upper_thresh=95, blue_lower_thresh=160) & \
               filter_blue(rgb, red_upper_thresh=30, green_upper_thresh=65, blue_lower_thresh=130) & \
               filter_blue(rgb, red_upper_thresh=130, green_upper_thresh=155, blue_lower_thresh=180) & \
               filter_blue(rgb, red_upper_thresh=40, green_upper_thresh=35, blue_lower_thresh=85) & \
               filter_blue(rgb, red_upper_thresh=30, green_upper_thresh=20, blue_lower_thresh=65) & \
               filter_blue(rgb, red_upper_thresh=90, green_upper_thresh=90, blue_lower_thresh=140) & \
               filter_blue(rgb, red_upper_thresh=60, green_upper_thresh=60, blue_lower_thresh=120) & \
               filter_blue(rgb, red_upper_thresh=110, green_upper_thresh=110, blue_lower_thresh=175)
    if output_type == "bool":
        pass
    elif output_type == "float":
        result = result.astype(float)
    else:
        result = result.astype("uint8") * 255
    return result

In [86]:
def filter_remove_small_objects(np_img, min_size=3000, avoid_overmask=True, overmask_thresh=95, output_type="uint8"):
    rem_sm = np_img.astype(bool)  
    rem_sm = sk_morphology.remove_small_objects(rem_sm, min_size=min_size)
    mask_percentage = mask_percent(rem_sm)
    if (mask_percentage >= overmask_thresh) and (min_size >= 1) and (avoid_overmask is True):
        new_min_size = min_size / 2
        print("Mask percentage %3.2f%% >= overmask threshold %3.2f%% for Remove Small Objs size %d, so try %d" % (mask_percentage, overmask_thresh, min_size, new_min_size))
        rem_sm = filter_remove_small_objects(np_img, new_min_size, avoid_overmask, overmask_thresh, output_type)
    np_img = rem_sm

    if output_type == "bool":
        pass
    elif output_type == "float":
        np_img = np_img.astype(float)
    else:
        np_img = np_img.astype("uint8") * 255
    return np_img


In [54]:
def mask_rgb(rgb, mask):
    result = rgb * np.dstack([mask, mask, mask])
    return result

In [ ]:
def filter_green_channel(np_img, green_thresh=200, avoid_overmask=True, overmask_thresh=90, output_type="bool"):
    g = np_img[:, :, 1]
    gr_ch_mask = (g < green_thresh) & (g > 0)
    mask_percentage = mask_percent(gr_ch_mask)
    if (mask_percentage >= overmask_thresh) and (green_thresh < 255) and (avoid_overmask is True):
        new_green_thresh = math.ceil((255 - green_thresh) / 2 + green_thresh)
        print("Mask percentage %3.2f%% >= overmask threshold %3.2f%% for Remove Green Channel green_thresh=%d, so try %d" % (mask_percentage, overmask_thresh, green_thresh, new_green_thresh))
        gr_ch_mask = filter_green_channel(np_img, new_green_thresh, avoid_overmask, overmask_thresh, output_type)
    np_img = gr_ch_mask
    if output_type == "bool":
        pass
    elif output_type == "float":
        np_img = np_img.astype(float)
    else:
        np_img = np_img.astype("uint8") * 255
    return np_img

In [121]:
def apply_image_filters(np_img, slide_num=None, info=None, save=False, display=False):
    rgb = np_img
    mask_not_green = filter_green_channel(rgb)
    rgb_not_green = mask_rgb(rgb, mask_not_green)
    
    mask_not_gray = filter_grays(rgb)
    rgb_not_gray = mask_rgb(rgb, mask_not_gray)

    mask_no_red_pen = filter_red_pen(rgb)
    rgb_no_red_pen = mask_rgb(rgb, mask_no_red_pen)

    mask_no_green_pen = filter_green_pen(rgb)
    rgb_no_green_pen = mask_rgb(rgb, mask_no_green_pen)

    mask_no_blue_pen = filter_blue_pen(rgb)
    rgb_no_blue_pen = mask_rgb(rgb, mask_no_blue_pen)
    mask_gray_green_pens = mask_not_gray & mask_not_green & mask_no_red_pen & mask_no_green_pen & mask_no_blue_pen
    rgb_gray_green_pens = mask_rgb(rgb, mask_gray_green_pens)
    

    mask_remove_small = filter_remove_small_objects(mask_gray_green_pens, min_size=500, output_type="bool")
    rgb_remove_small = mask_rgb(rgb, mask_remove_small)
    not_greenish = filter_green(rgb_remove_small, red_upper_thresh=125, green_lower_thresh=30, blue_lower_thresh=30)
    not_grayish = filter_grays(rgb_remove_small, tolerance=30)
    rgb_new = mask_rgb(rgb, not_greenish & not_grayish)
    img = rgb_new
    return img

In [133]:
def apply_filters_to_image(slide_num, save=True, display=False):
    print("Processing slide #%d" % slide_num)
    if save and not os.path.exists(FILTER_DIR):
        os.makedirs(FILTER_DIR)
    img_path = get_training_image_path(slide_num)
    np_orig = open_image_np(img_path)
    filtered_np_img = apply_image_filters(np_orig, slide_num,save=save, display=display)
    if save:
        result_path = get_filter_image_path(slide_num)
        pil_img = np_to_pil(filtered_np_img)
        pil_img.save(result_path)
    return filtered_np_img


In [127]:
def apply_filters_to_image_range(start_ind, end_ind, save, display):
    for slide_num in range(start_ind, end_ind):
        _= apply_filters_to_image(slide_num, save=save, display=display)
    return start_ind, end_ind

In [138]:
def multiprocess_apply_filters_to_images(save=True, display=False):
    print("Applying filters to images (multiprocess)\n")

    if save and not os.path.exists(FILTER_DIR):
        os.makedirs(FILTER_DIR)


    num_processes = multiprocessing.cpu_count()
    pool = multiprocessing.Pool(num_processes)
    num_train_images = len(files)
    if num_processes > num_train_images:
        num_processes = num_train_images
    images_per_process = num_train_images / num_processes

    print("Number of processes: " + str(num_processes))
    print("Number of training images: " + str(num_train_images))

    tasks = []
    for num_process in range(1, num_processes + 1):
        start_index = (num_process - 1) * images_per_process
        end_index = num_process * images_per_process
        start_index = int(start_index)
        end_index = int(end_index)
        tasks.append((start_index, end_index, save, display))
    if start_index == end_index:
        print("Task #" + str(num_process) + ": Process slide " + str(start_index))
    else:
        print("Task #" + str(num_process) + ": Process slides " + str(start_index) + " to " + str(end_index))

  
    results = []
    for t in tasks:
        results.append(pool.apply_async(apply_filters_to_image_range, t))


In [24]:
def get_filter_image_path(slide_number):
    dir = FILTER_DIR
    if not os.path.exists(dir):
        os.makedirs(dir)
    img_path = os.path.join(dir, get_train_prefix(slide_number) + "." + DEST_TRAIN_EXT)
    return img_path

'filter_png/TCGA-61-2110-01A-01-TS1.png'

In [89]:
def get_tile_data_path(slide_number):
    dir = TILE_DATA_DIR
    if not os.path.exists(dir):
        os.makedirs(dir)
    img_path = os.path.join(dir, get_train_prefix(slide_number))
    return img_path

In [29]:
def get_image_dimensions(slide_num):
        slide = open_slide(slide_num)
        large_w, large_h = slide.dimensions
        new_w = math.floor(large_w / SCALE_FACTOR)
        new_h = math.floor(large_h / SCALE_FACTOR)
        return large_w, large_h, new_w, new_h

In [33]:
def get_num_tiles(rows, cols, row_tile_size, col_tile_size):
    num_row_tiles = math.ceil(rows / row_tile_size)
    num_col_tiles = math.ceil(cols / col_tile_size)
    return num_row_tiles, num_col_tiles

In [34]:
def get_tile_indices(rows, cols, row_tile_size, col_tile_size):
    indices = list()
    num_row_tiles, num_col_tiles = get_num_tiles(rows, cols, row_tile_size, col_tile_size)
    for r in range(0, num_row_tiles):
        start_r = r * row_tile_size
        end_r = ((r + 1) * row_tile_size) if (r < num_row_tiles - 1) else rows
        for c in range(0, num_col_tiles):
            start_c = c * col_tile_size
            end_c = ((c + 1) * col_tile_size) if (c < num_col_tiles - 1) else cols
            indices.append((start_r, end_r, start_c, end_c, r + 1, c + 1))
    return indices

In [35]:
def tissue_percent(np_img):
    return 100 - mask_percent(np_img)

In [77]:
def tissue_quantity(tissue_percentage):
    if tissue_percentage >= TISSUE_HIGH_THRESH:
        return TissueQuantity.HIGH
    elif (tissue_percentage >= TISSUE_LOW_THRESH) and (tissue_percentage < TISSUE_HIGH_THRESH):
        return TissueQuantity.MEDIUM
    elif (tissue_percentage > 0) and (tissue_percentage < TISSUE_LOW_THRESH):
        return TissueQuantity.LOW
    else:
        return TissueQuantity.NONE

In [37]:
def small_to_large_mapping(small_pixel, large_dimensions):
    small_x, small_y = small_pixel
    large_w, large_h = large_dimensions
    large_x = round((large_w / SCALE_FACTOR) / math.floor(large_w / SCALE_FACTOR) * (SCALE_FACTOR * small_x))
    large_y = round((large_h / SCALE_FACTOR) / math.floor(large_h / SCALE_FACTOR) * (SCALE_FACTOR * small_y))
    return large_x, large_y

In [43]:
def filter_rgb_to_hsv(np_img, display_np_info=True):
    hsv = sk_color.rgb2hsv(np_img)
    return hsv

In [44]:
def filter_hsv_to_h(hsv, output_type="int", display_np_info=True):
    h = hsv[:, :, 0]
    h = h.flatten()
    if output_type == "int":
        h *= 360
        h = h.astype("int")
    return h

In [45]:
def rgb_to_hues(rgb):
    hsv = filter_rgb_to_hsv(rgb, display_np_info=False)
    h = filter_hsv_to_h(hsv, display_np_info=False)
    return h

In [47]:
def hsv_purple_deviation(hsv_hues):
    purple_deviation = np.sqrt(np.mean(np.abs(hsv_hues - HSV_PURPLE) ** 2))
    return purple_deviation

In [48]:
def hsv_pink_deviation(hsv_hues):
    pink_deviation = np.sqrt(np.mean(np.abs(hsv_hues - HSV_PINK) ** 2))
    return pink_deviation

In [50]:
def hsv_purple_pink_factor(rgb):
    hues = rgb_to_hues(rgb)
    hues = hues[hues >= 260]  
    hues = hues[hues <= 340]  
    if len(hues) == 0:
        return 0 
    pu_dev = hsv_purple_deviation(hues)
    pi_dev = hsv_pink_deviation(hues)
    avg_factor = (340 - np.average(hues)) ** 2
    if pu_dev == 0:  
        return 0

    factor = pi_dev / pu_dev * avg_factor
    return factor

In [51]:
def filter_hsv_to_s(hsv):
    s = hsv[:, :, 1]
    s = s.flatten()
    return s

In [52]:
def filter_hsv_to_v(hsv):
    v = hsv[:, :, 2]
    v = v.flatten()
    return v

In [54]:
def hsv_saturation_and_value_factor(rgb):
    hsv = filter_rgb_to_hsv(rgb, display_np_info=False)
    s = filter_hsv_to_s(hsv)
    v = filter_hsv_to_v(hsv)
    s_std = np.std(s)
    v_std = np.std(v)
    if s_std < 0.05 and v_std < 0.05:
        factor = 0.4
    elif s_std < 0.05:
        factor = 0.7
    elif v_std < 0.05:
        factor = 0.7
    else:
        factor = 1

    factor = factor ** 2
    return factor

In [55]:
def tissue_quantity_factor(amount):
    if amount == TissueQuantity.HIGH:
        quantity_factor = 1.0
    elif amount == TissueQuantity.MEDIUM:
        quantity_factor = 0.2
    elif amount == TissueQuantity.LOW:
        quantity_factor = 0.1
    else:
        quantity_factor = 0.0
    return quantity_factor

In [38]:
def score_tile(np_tile, tissue_percent, slide_num, row, col):
    color_factor = hsv_purple_pink_factor(np_tile)
    s_and_v_factor = hsv_saturation_and_value_factor(np_tile)
    amount = tissue_quantity(tissue_percent)
    quantity_factor = tissue_quantity_factor(amount)
    combined_factor = color_factor * s_and_v_factor * quantity_factor
    score = (tissue_percent ** 2) * np.log(1 + combined_factor) / 1000.0
    score = 1.0 - (10.0 / (10.0 + score))
    return score, color_factor, s_and_v_factor, quantity_factor

In [84]:
def score_tiles(slide_num, np_img=None, dimensions=None, small_tile_in_tile=False):
    if dimensions is None:
        img_path = get_filter_image_path(slide_num)
        o_w, o_h, w, h = get_image_dimensions(slide_num)
    else:
        o_w, o_h, w, h = dimensions

    if np_img is None:
        np_img = open_image_np(img_path)

    row_tile_size = round(ROW_TILE_SIZE / SCALE_FACTOR)  
    col_tile_size = round(COL_TILE_SIZE / SCALE_FACTOR)  

    num_row_tiles, num_col_tiles = get_num_tiles(h, w, row_tile_size, col_tile_size)

    tile_sum = TileSummary(slide_num=slide_num,
                         orig_w=o_w,
                         orig_h=o_h,
                         orig_tile_w=COL_TILE_SIZE,
                         orig_tile_h=ROW_TILE_SIZE,
                         scaled_w=w,
                         scaled_h=h,
                         scaled_tile_w=col_tile_size,
                         scaled_tile_h=row_tile_size,
                         tissue_percentage=tissue_percent(np_img),
                         num_col_tiles=num_col_tiles,
                         num_row_tiles=num_row_tiles)

    count = 0
    high = 0
    medium = 0
    low = 0
    none = 0
    tile_indices = get_tile_indices(h, w, row_tile_size, col_tile_size)
    for t in tile_indices:
        count += 1  
        r_s, r_e, c_s, c_e, r, c = t
        np_tile = np_img[r_s:r_e, c_s:c_e]
        t_p = tissue_percent(np_tile)
        amount = tissue_quantity(t_p)
        if amount == TissueQuantity.HIGH:
            high += 1
        elif amount == TissueQuantity.MEDIUM:
            medium += 1
        elif amount == TissueQuantity.LOW:
            low += 1
        elif amount == TissueQuantity.NONE:
            none += 1
        o_c_s, o_r_s = small_to_large_mapping((c_s, r_s), (o_w, o_h))
        o_c_e, o_r_e = small_to_large_mapping((c_e, r_e), (o_w, o_h))
        if (o_c_e - o_c_s) > COL_TILE_SIZE:
            o_c_e -= 1
        if (o_r_e - o_r_s) > ROW_TILE_SIZE:
            o_r_e -= 1

        score, color_factor, s_and_v_factor, quantity_factor = score_tile(np_tile, t_p, slide_num, r, c)

        np_scaled_tile = np_tile if small_tile_in_tile else None
        tile = Tile(tile_sum, slide_num, np_scaled_tile, count, r, c, r_s, r_e, c_s, c_e, o_r_s, o_r_e, o_c_s,
                o_c_e, t_p, color_factor, s_and_v_factor, quantity_factor, score)
        tile_sum.tiles.append(tile)

    tile_sum.count = count
    tile_sum.high = high
    tile_sum.medium = medium
    tile_sum.low = low
    tile_sum.none = none

    tiles_by_score = tile_sum.tiles_by_score()
    rank = 0
    for t in tiles_by_score:
        rank += 1
        t.rank = rank

    return tile_sum

In [86]:
def save_tile_data(tile_summary):
    csv = summary_title(tile_summary) + "\n" + summary_stats(tile_summary)

    csv += "\n\n\nTile Num,Row,Column,Tissue %,Tissue Quantity,Col Start,Row Start,Col End,Row End,Col Size,Row Size," + \
             "Original Col Start,Original Row Start,Original Col End,Original Row End,Original Col Size,Original Row Size," + \
             "Color Factor,S and V Factor,Quantity Factor,Score\n"

    for t in tile_summary.tiles:
        line = "%d,%d,%d,%4.2f,%s,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%d,%4.0f,%4.2f,%4.2f,%0.4f\n" % (
          t.tile_num, t.r, t.c, t.tissue_percentage, t.tissue_quantity().name, t.c_s, t.r_s, t.c_e, t.r_e, t.c_e - t.c_s,
          t.r_e - t.r_s, t.o_c_s, t.o_r_s, t.o_c_e, t.o_r_e, t.o_c_e - t.o_c_s, t.o_r_e - t.o_r_s, t.color_factor,
          t.s_and_v_factor, t.quantity_factor, t.score)
        csv += line

    data_path = get_tile_data_path(tile_summary.slide_num)
    csv_file = open(data_path, "w")
    csv_file.write(csv)
    csv_file.close()

In [91]:
def tile_to_pil_tile(tile):
    t = tile
    s = open_slide(t.slide_num)

    x, y = t.o_c_s, t.o_r_s
    w, h = t.o_c_e - t.o_c_s, t.o_r_e - t.o_r_s
    tile_region = s.read_region((x, y), 0, (w, h))
    pil_img = tile_region.convert("RGB")
    return pil_img

In [104]:
def get_tile_image_path(t,slide_num):
    tile_path = os.path.join(TILE_DIR, get_train_prefix(slide_num) + "-r%d-c%d-x%d-y%d-w%d-h%d" % (
                             t.r, t.c, t.o_c_s, t.o_r_s, t.o_c_e - t.o_c_s, t.o_r_e - t.o_r_s) + "." + DEST_TRAIN_EXT)
    return tile_path

In [98]:
def save_display_tile(tile, save=True, display=False):
    tile_pil_img = tile_to_pil_tile(tile)
    if save:
        img_path = get_tile_image_path(tile,tile.slide_num)
        dir = os.path.dirname(img_path)
    if not os.path.exists(dir):
        os.makedirs(dir)
    tile_pil_img.save(img_path)

    if display:
        tile_pil_img.show()

In [107]:
def summary_and_tiles(slide_num, display=True, save_summary=False, save_data=False, save_top_tiles=True):
    img_path = get_filter_image_path(slide_num)
    np_img = open_image_np(img_path)

    tile_sum = score_tiles(slide_num, np_img)
    if save_data:
        save_tile_data(tile_sum)
        generate_tile_summaries(tile_sum, np_img, display=display, save_summary=save_summary)
        generate_top_tile_summaries(tile_sum, np_img, display=display, save_summary=save_summary)
    if save_top_tiles:
        for tile in tile_sum.top_tiles():
            save_tile(tile)
    return tile_sum

In [111]:
def image_range_to_tiles(start_ind, end_ind, display=False, save_summary=False, save_data=False, save_top_tiles=True):
    image_num_list = list()
    tile_summaries_dict = dict()
    for slide_num in range(start_ind, end_ind):
        tile_summary = summary_and_tiles(slide_num, display, save_summary, save_data, save_top_tiles)
        image_num_list.append(slide_num)
        tile_summaries_dict[slide_num] = tile_summary
    return image_num_list, tile_summaries_dict

In [114]:
def multiprocess_filtered_images_to_tiles(display=False, save_summary=False, save_data=False, save_top_tiles=True,
                                          html=False, image_num_list=None):
    print("Generating tile summaries (multiprocess)\n")

    if save_summary and not os.path.exists(TILE_SUMMARY_DIR):
        os.makedirs(TILE_SUMMARY_DIR)

    num_processes = multiprocessing.cpu_count()
    pool = multiprocessing.Pool(num_processes)

    if image_num_list is not None:
        num_train_images = len(image_num_list)
    else:
        num_train_images = len(files)
    if num_processes > num_train_images:
        num_processes = num_train_images
    images_per_process = num_train_images / num_processes

    print("Number of processes: " + str(num_processes))
    print("Number of training images: " + str(num_train_images))

    tasks = []
    for num_process in range(1, num_processes + 1):
        start_index = (num_process - 1) * images_per_process 
        end_index = num_process * images_per_process
        start_index = int(start_index)
        end_index = int(end_index)
        if image_num_list is not None:
            sublist = image_num_list[start_index - 1:end_index]
            tasks.append((sublist, display, save_summary, save_data, save_top_tiles))
            print("Task #" + str(num_process) + ": Process slides " + str(sublist))
        else:
            tasks.append((start_index, end_index, display, save_summary, save_data, save_top_tiles))
        if start_index == end_index:
            print("Task #" + str(num_process) + ": Process slide " + str(start_index))
        else:
            print("Task #" + str(num_process) + ": Process slides " + str(start_index) + " to " + str(end_index))

      # start tasks
    results = []
    for t in tasks:
        if image_num_list is not None:
            results.append(pool.apply_async(image_list_to_tiles, t))
        else:
            results.append(pool.apply_async(image_range_to_tiles, t))

    slide_nums = list()
    tile_summaries_dict = dict()
    for result in results:
        image_nums, tile_summaries = result.get()
        slide_nums.extend(image_nums)
        tile_summaries_dict.update(tile_summaries)
        print("Done tiling slides: %s" % image_nums)


In [115]:
multiprocess_filtered_images_to_tiles()

Generating tile summaries (multiprocess)

Number of processes: 8
Number of training images: 1368
Task #1: Process slides 0 to 171
Task #2: Process slides 171 to 342
Task #3: Process slides 342 to 513
Task #4: Process slides 513 to 684
Task #5: Process slides 684 to 855
Task #6: Process slides 855 to 1026
Task #7: Process slides 1026 to 1197
Task #8: Process slides 1197 to 1368
Done tiling slides: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141,

In [75]:
class TileSummary:
  

    slide_num = None
    orig_w = None
    orig_h = None
    orig_tile_w = None
    orig_tile_h = None
    scale_factor = SCALE_FACTOR
    scaled_w = None
    scaled_h = None
    scaled_tile_w = None
    scaled_tile_h = None
    mask_percentage = None
    num_row_tiles = None
    num_col_tiles = None

    count = 0
    high = 0
    medium = 0
    low = 0
    none = 0

    def __init__(self, slide_num, orig_w, orig_h, orig_tile_w, orig_tile_h, scaled_w, scaled_h, scaled_tile_w,
                   scaled_tile_h, tissue_percentage, num_col_tiles, num_row_tiles):
        self.slide_num = slide_num
        self.orig_w = orig_w
        self.orig_h = orig_h
        self.orig_tile_w = orig_tile_w
        self.orig_tile_h = orig_tile_h
        self.scaled_w = scaled_w
        self.scaled_h = scaled_h
        self.scaled_tile_w = scaled_tile_w
        self.scaled_tile_h = scaled_tile_h
        self.tissue_percentage = tissue_percentage
        self.num_col_tiles = num_col_tiles
        self.num_row_tiles = num_row_tiles
        self.tiles = []

    def __str__(self):
        return summary_title(self) + "\n" + summary_stats(self)

    def mask_percentage(self):
        return 100 - self.tissue_percentage

    def num_tiles(self):
        return self.num_row_tiles * self.num_col_tiles

    def tiles_by_tissue_percentage(self):
        sorted_list = sorted(self.tiles, key=lambda t: t.tissue_percentage, reverse=True)
        return sorted_list

    def tiles_by_score(self):
        sorted_list = sorted(self.tiles, key=lambda t: t.score, reverse=True)
        return sorted_list

    def top_tiles(self):
        sorted_tiles = self.tiles_by_score()
        top_tiles = sorted_tiles[:NUM_TOP_TILES]
        return top_tiles

    def get_tile(self, row, col):
        tile_index = (row - 1) * self.num_col_tiles + (col - 1)
        tile = self.tiles[tile_index]
        return tile

    def display_summaries(self):
        summary_and_tiles(self.slide_num, display=True, save_summary=False, save_data=False, save_top_tiles=False)

In [74]:
class Tile:
    def __init__(self, tile_summary, slide_num, np_scaled_tile, tile_num, r, c, r_s, r_e, c_s, c_e, o_r_s, o_r_e, o_c_s,
                 o_c_e, t_p, color_factor, s_and_v_factor, quantity_factor, score):
        self.tile_summary = tile_summary
        self.slide_num = slide_num
        self.np_scaled_tile = np_scaled_tile
        self.tile_num = tile_num
        self.r = r
        self.c = c
        self.r_s = r_s
        self.r_e = r_e
        self.c_s = c_s
        self.c_e = c_e
        self.o_r_s = o_r_s
        self.o_r_e = o_r_e
        self.o_c_s = o_c_s
        self.o_c_e = o_c_e
        self.tissue_percentage = t_p
        self.color_factor = color_factor
        self.s_and_v_factor = s_and_v_factor
        self.quantity_factor = quantity_factor
        self.score = score

    def __str__(self):
        return "[Tile #%d, Row #%d, Column #%d, Tissue %4.2f%%, Score %0.4f]" % (
          self.tile_num, self.r, self.c, self.tissue_percentage, self.score)

    def __repr__(self):
        return "\n" + self.__str__()

    def mask_percentage(self):
        return 100 - self.tissue_percentage

    def tissue_quantity(self):
        return tissue_quantity(self.tissue_percentage)

    def get_pil_tile(self):
        return tile_to_pil_tile(self)

    def get_np_tile(self):
        return tile_to_np_tile(self)

    def save_tile(self):
        save_display_tile(self, save=True, display=False)

    def display_tile(self):
        save_display_tile(self, save=False, display=True)

    def display_with_histograms(self):
        display_tile(self, rgb_histograms=True, hsv_histograms=True)

    def get_np_scaled_tile(self):
        return self.np_scaled_tile

    def get_pil_scaled_tile(self):
        return np_to_pil(self.np_scaled_tile)

In [65]:
class TissueQuantity(Enum):
    NONE = 0
    LOW = 1
    MEDIUM = 2
    HIGH = 3